# OmniMarket ML Trainer
Train machine learning models to predict:
- Option premium (price)
- P&L at expiry
- Implied volatility

In [ ]:
!pip install yfinance scikit-learn

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
# Step 1: Download underlying price data
ticker = input("Enter stock ticker: ").upper()
end_date = datetime.today()
start_date = end_date - timedelta(days=180)
stock_data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
stock_data = stock_data[['Close']]
stock_data.head()

In [ ]:
# Step 2: Pick one expiration and build training data from chain
yf_ticker = yf.Ticker(ticker)
expirations = yf_ticker.options
print("Available expirations:", expirations)
expiry = input("Choose one expiration date: ")
chain = yf_ticker.option_chain(expiry)
calls = chain.calls
puts = chain.puts

In [ ]:
# Step 3: Feature engineering
def create_features(df, option_type):
    df = df.copy()
    df['type'] = 1 if option_type == 'call' else 0
    df['moneyness'] = df['strike'] / stock_data['Close'][-1]
    df['mid_price'] = (df['bid'] + df['ask']) / 2
    df['iv'] = df.get('impliedVolatility', 0.3)
    df = df[['strike', 'lastPrice', 'moneyness', 'mid_price', 'iv', 'type']].dropna()
    return df

In [ ]:
# Prepare features and combine
call_features = create_features(calls, 'call')
put_features = create_features(puts, 'put')
df = pd.concat([call_features, put_features], ignore_index=True)
df = df.dropna()
df.head()

In [ ]:
# Step 4: Train-test split and model training
X = df[['strike', 'moneyness', 'iv', 'type']]
y = df['mid_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Step 5: Evaluation
y_pred = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))

In [ ]:
# Use the model to predict new data (example)
sample_input = X_test.iloc[:5]
print("Predictions:", model.predict(sample_input))